In [6]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib

In [55]:
# data_dir = pathlib.Path("dataset/train")
# image_count = len(list(data_dir.glob('*/*.png')))
# benign = list(data_dir.glob('0/*'))
# malicious = list(data_dir.glob('1/*'))

# data_array = np.empty((0, 2))
# for ben in benign:
#     ar=np.asarray(PIL.Image.open(ben))
#     data_array = np.vstack((data_array, np.array([ar, 0])))

    
# for mal in malicious:
#     ar=np.asarray(PIL.Image.open(mal))
#     data_array = np.vstack((data_array, np.array([ar, 1])))

# np.save("converted_dataset", np.array(data_array))

# data_dir = pathlib.Path("dataset/test")
# image_count = len(list(data_dir.glob('*/*.png')))
# benign = list(data_dir.glob('0/*'))
# malicious = list(data_dir.glob('1/*'))

# data_array = np.empty((0, 2))
# for ben in benign:
#     ar=np.asarray(PIL.Image.open(ben))
#     data_array = np.vstack((data_array, np.array([ar, 0])))

    
# for mal in malicious:
#     ar=np.asarray(PIL.Image.open(mal))
#     data_array = np.vstack((data_array, np.array([ar, 1])))

# np.save("converted_test_dataset", np.array(data_array))

In [49]:
dataset = np.load("converted_dataset.npy",allow_pickle=True)
np.random.shuffle(dataset)

In [50]:
N = np.shape(data_array)[0]

x_train = dataset[:N, 0]
y_train = dataset[:N, 1]



In [53]:
print(x_train[0].shape)

(28, 28)
